In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sys

In [ ]:
def handle_request(p_url, p_method="GET", p_params=None, p_headers=None):
    try:
        response = requests.request(
            method=p_method,
            url=p_url,
            params=p_params,
            headers=p_headers,
        )
        # print(response.url)
        if response.status_code == 200:
            # return response.content
            return response.json()
        else:
            print(response.status_code)
            return response.status_code
    except:
        print(-1)
        return -1


def extract_info_from_page(json_data):
    my_list = []
    for data in json_data["data"]["products"]:
        if data["status"] == "out_of_stock":
            return False, pd.DataFrame(my_list)
        temp = {
            "id": data["id"],
            "title_en": data["title_en"],
            "title_fa": data["title_fa"],
            "price": data["default_variant"]["price"]["rrp_price"],
            "selling_price": data["default_variant"]["price"]["selling_price"],
            # "order_limit": data["default_variant"]["price"]["order_limit"],
            # "marketable_stock": data["default_variant"]["price"]["marketable_stock"],
            "discount": data["default_variant"]["price"]["discount_percent"],
            "min_price_in_last_month": data["properties"]["min_price_in_last_month"],
        }
        my_list.append(temp)
    return True, pd.DataFrame(my_list)


def create_product_df(main_url, params, key_param):
    page = 1
    final_df = None
    while True:
        params[key_param] = str(page)
        json_data = handle_request(main_url, p_params=params)

        progress_completed = f"page {page} completed."
        sys.stdout.write("\r" + progress_completed)
        sys.stdout.flush()
        page += 1

        sw, df = extract_info_from_page(json_data)
        final_df = pd.concat([final_df, df], ignore_index=True)
        if sw == False:
            return final_df

## Init Variables

In [ ]:
main_url = "https://api.digikala.com/v1/categories/"

key_param1 = "seller_types[0]"
val_param1 = "digikala"

key_param2 = "seo_url"

key_param3 = "page"
val_param3 = "1"

## Phone

In [ ]:
category_url = "mobile-phone/search/"
val_param2 = "/category-mobile-phone/product-list/?seller_types%5B0%5D=digikala"
params = {
    key_param1: val_param1,
    key_param2: val_param2,
    key_param3: val_param3,
}
phone_df = create_product_df(main_url + category_url, params, key_param3)

In [ ]:
phone_df

## Laptop

In [ ]:
category_url = "notebook-netbook-ultrabook/search/"
val_param2 = "/notebook-netbook-ultrabook/?seller_types%5B0%5D=digikala"
params = {
    key_param1: val_param1,
    key_param2: val_param2,
    key_param3: val_param3,
}
laptop_df = create_product_df(main_url + category_url, params, key_param3)

In [ ]:
laptop_df

## Hint

```python
data["data"]["products"][0]["id"]
data["data"]["products"][0]["title_en"]
data["data"]["products"][0]["title_fa"]
data["data"]["products"][0]["status"]
data["data"]["products"][0]["default_variant"]["price"]
data["data"]["products"][0]["default_variant"]["price"]["selling_price"]
data["data"]["products"][0]["default_variant"]["price"]["rrp_price"]
data["data"]["products"][0]["default_variant"]["price"]["order_limit"]
data["data"]["products"][0]["default_variant"]["price"]["marketable_stock"]
data["data"]["products"][0]["default_variant"]["price"]["discount_percent"]
data["data"]["products"][0]["properties"]["min_price_in_last_month"]
```